In [1]:
from pathlib import Path
from collections import defaultdict
import numpy as np

data_dir = Path("data/")
dataset = "EN" # EN or ES

# Part 1(i): Emission scores
Compute $e(x|y) = \frac{\text{Count}(y \rightarrow x)}{\text{Count}(y)}$ and store it into a dictionary $f$.

In [18]:
def calculate_emission_scores(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        f (dict): Key-value mapping of str(x, y) to log(e(x|y)) values.
    '''

    count_emission = defaultdict(int) # Stores Count(y -> x), where key is tuple (x, y), and value is Count(y -> x)
    count_labels = defaultdict(int) # Stores Count(y), where key is y
    
    # For computing pairs that are not aligned
    unique_x = set()
    unique_y = set()
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]

                count_emission[(x, y)] += 1
                count_labels[y] += 1
                
                unique_x.add(x)
                unique_y.add(y)
    
    # Result dictionary that maps str(x, y) to log(e(x|y)) values
    f = {}
    
    # Estimate e(x|y) based on counts
    for x, y in count_emission:
        # Create str(x, y)
        feature_str = f"emission:{y}+{x}"
        e = count_emission[(x, y)] / count_labels[y]
        
        # Store score
        f[feature_str] = np.log(e)
    
    # Set features for emissions that are not aligned
    for x in unique_x:
        for y in unique_y:
            feature_str = f"emission:{y}+{x}"
            
            # Only set to -inf if it doesn't exist as an aligned pair
            if feature_str not in f:
                f[feature_str] = -10**8
    
    return f

# Part 1(ii): Transition scores
Compute $q(y_i|y_{i-1}) = \frac{\text{Count}(y_{i-1}, y_i)}{\text{Count}(y_{i-1})}$ and store it into a dictionary $f$.

In [19]:
def calculate_transition_scores(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        f (dict): Key-value mapping of str(y_{i-1}, y_i) to log(q(y_i|y_{i-1})) values.
    '''
    
    count_transition = defaultdict(int) # Key is tuple (y_i, y_{i-1}), and value is Count(y_{i-1}, y_i)
    count_labels = defaultdict(int) # Stores Count(y_i), where key is y_i
    
    # For computing pairs that are not aligned
    unique_y = set(["START", "STOP"])

    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        # Initialize prev_y as START
        prev_y = "START"
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]
                
                transition_key = (prev_y, y)
                count_transition[transition_key] += 1
                count_labels[y] += 1
                
                # Update for next word
                prev_y = y
                
                unique_y.add(y)
            else:
                # End of sentence
                # Store Count(STOP|y_n)
                transition_key = (prev_y, "STOP")
                count_transition[transition_key] += 1
                
                # Start of next sentence: initialize prev_y to "START" and store Count(START)
                prev_y = "START"
                count_labels[prev_y] += 1
    
    # Result dictionary that maps str(x, y) to log(e(x|y)) values
    f = {}
    
    # Estimate e(x|y) based on counts
    for prev_y, y in count_transition:
        # Create str(y_{i-1}, y_i)
        feature_str = f"transition:{prev_y}+{y}"
        q = count_transition[(prev_y, y)] / count_labels[prev_y]
        
        # Store score
        f[feature_str] = np.log(q)
    
    # Set features for transitions that are not aligned
    for prev_y in unique_y:
        for y in unique_y:
            feature_str = f"transition:{prev_y}+{y}"
            
            # Only set to -inf if it doesn't exist as an aligned pair
            if feature_str not in f:
                f[feature_str] = -10**8

    return f

In [20]:
# Compute emission and transition parameters
f_emission_train = calculate_emission_scores(data_dir / dataset / "train")
f_transition_train = calculate_transition_scores(data_dir / dataset / "train")

# Combine the transition and emission dictionaries together
f = {**f_emission_train, **f_transition_train}
# Ensure the number of elements is correct
assert(len(f) == len(f_emission_train) + len(f_transition_train))

# Part 2(i)
Compute CRF scores for a given input and output sequence pair.

In [21]:
def compute_crf_score(x, y, feature_dict):
    ''' 
    Inputs:
        x (list[str]): Complete input word sentence (without START or STOP tags)
        y (list[str]): Complete output label sequence
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        p (float): Score given by p(y | x)
    '''
    
    # Input and output sequences must be of the same length
    assert(len(x) == len(y))
    n = len(x) # Sequence length
    
    # Stores the number of times each feature appears in (x, y)
    feature_count = defaultdict(int)
    
    # Compute emission features
    for i in range(n):
        formatted_word = x[i]
        emission_key = f"emission:{y[i]}+{formatted_word}"
        feature_count[emission_key] += 1
    
    # Compute transition features
    # Add START and STOP tags to y
    updated_y = ["START"] + y + ["STOP"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = f"transition:{prev_y}+{y_i}"
        feature_count[transition_key] += 1
    
    # Compute score
    score = 0
    for feature_key, count in feature_count.items():
        weight = feature_dict[feature_key]
        score += weight * count
    
    return score

# # Test function
# x = "Great food with an awesome atmosphere !".split()
# y = "O B-positive O O O B-positive O".split()
# compute_crf_score(x, y, f)

# Part 2(ii)
Viterbi algorithm for decoding.

In [22]:
def get_states(path):
    '''
    Inputs:
        path (Path object or str): path on the local directory to the dataset to load from.
    Outputs:
        states (list[str]): Unique states in the dataset.
    '''
    states = set()
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        
        # Process lines
        for line in lines:
            # Strip newline
            formatted_line = line.strip()
            
            # Only process lines that are not newlines
            if len(formatted_line) > 0:
                # Split into (x, y) pair
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]
                states.add(y)
    
    return list(states)

def viterbi_decode(x, states, feature_dict, default_index=0):
    '''
    Inputs:
        x (list[str]): Input sequence.
        states (list[str]): Possible output states.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        default_index (int, optional): Index to default for backpointer if scores are nan.
    Outputs:
        y (list[str]): Most probable output sequence.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    scores = np.full((n, d), -np.inf) # Default state is -inf for missing values
    bp = np.full((n, d), default_index, dtype=np.int) # Set default backpointer to the default_index state

    # Convert to lowercase
    x = [x[i] for i in range(n)]
    
    # Compute START transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:START+{current_y}"
        emission_key = f"emission:{current_y}+{x[0]}"
        transmission_score = feature_dict.get(transition_key, -10**8)
        emission_score = feature_dict.get(emission_key, -10**8)
        scores[0, i] = transmission_score + emission_score
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, prev_y in enumerate(states):
            for j, current_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                transition_score = feature_dict.get(transition_key, -10**8)
                emission_score = feature_dict.get(emission_key, -10**8)
                overall_score = emission_score + transition_score + scores[i-1, k]

                # Better score is found: Update backpointer and score arrays
                if overall_score > scores[i, j]:
                    scores[i, j] = overall_score
                    bp[i,j] = k
    
    # Compute for STOP
    highest_score = None
    highest_bp = default_index
    
    for j, prev_y in enumerate(states):
        transition_key = f"transition:{prev_y}+STOP"
        transition_score = feature_dict.get(transition_key, -10**8)
        overall_score = transition_score + scores[n-1, j]
        
        if highest_score == None or overall_score > highest_score:
            highest_score = overall_score
            highest_bp = j
    
    # Follow backpointers to get output sequence
    result = [states[highest_bp]]
    prev_bp = highest_bp
    for i in range(n-1, 0, -1):
        prev_bp = bp[i, prev_bp]
        output = states[prev_bp]
        # Prepend result to output list
        result = [output] + result
    
    return result

states = get_states(data_dir / dataset / "train")
# viterbi_decode("Great food with an awesome atmosphere !".split(), states, f) # Should be similar or equal to y

In [7]:
# Perform decoding on dev sets
def inference(path, states, feature_dict,output_type):
    '''
    Given a path, perform inference on sentences in the dataset and writes it to disk.
    
    Inputs:
        path (Path object or str): path on the local directory to the dataset to perform inference on.
        states (list[str]): Unique states that can be predicted.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        None
    '''
    
    default_index = states.index('O')
    sentences = []

    # Write predictions to file
    output_filename = str(path).replace(".in", ".{}.out".format(output_type))

    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        
        for line in lines:
            formatted_line = line.strip()
            
            # Not the end of sentence, add it to the list
            if len(formatted_line) > 0:
                sentence.append(formatted_line)
            else:
                # End of sentence
                sentences.append(sentence)
                sentence = []
    
    # Write output file
    with open(output_filename, "w") as wf:
        for sentence in sentences:
            # Run predictions
            pred_sentence = viterbi_decode(sentence, states, feature_dict, default_index)
            
            # Write original word and predicted tags
            for i in range(len(sentence)):
                wf.write(sentence[i] + " " + pred_sentence[i] + "\n")
            
            # End of sentence, write newline
            wf.write("\n")

inference(data_dir / dataset / "dev.in", states, f, output_type='p2')

# Part 3(i)
Loss calculation using forward algorithm. We first define the score calculation functions for a single sample.

In [8]:
def compute_forward_score(x, feature_dict, states):
    '''
    Uses the forward algorithm to compute the score for a given sequence.
    
    Inputs:
        x (list[str]): Input sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        forward_score (float): Forward score for this sequence.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    forward_scores = np.zeros((n, d))
    
    # Start forward pass
    # Compute START transition scores
    for i, current_y in enumerate(states):
        transition_key = f"transition:START+{current_y}"
        emission_key = f"emission:{current_y}+{x[0]}"
        transition_score = feature_dict.get(transition_key, -10**8)
        emission_score = feature_dict.get(emission_key, -10**8)
        # Sum exponentials
        forward_scores[0, i] = transition_score + emission_score
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, current_y in enumerate(states):
            temp_score = 0
            for j, prev_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                transition_score = feature_dict.get(transition_key, -10**8)
                emission_score = feature_dict.get(emission_key, -10**8)
                
                # Sum exponentials
                temp_score += np.exp(min(emission_score + transition_score + forward_scores[i-1, j], 700))

            # Add to forward scores array
            forward_scores[i, k] = np.log(temp_score) if temp_score else -10**8

    # Compute for STOP
    forward_prob = 0
    for j, prev_y in enumerate(states):
        transition_key = f"transition:{prev_y}+STOP"
        transition_score = feature_dict.get(transition_key, -10**8)
        # Sum exponentials
        overall_score = np.exp(min(transition_score + forward_scores[n-1, j], 700))
        forward_prob += overall_score
    # End forward pass
    
    alpha = np.log(forward_prob) if forward_prob else -700
    return forward_scores, alpha


def compute_crf_loss_sample(x, y, feature_dict, states):
    '''
    Inputs:
        x (list[str]): Input sequence.
        y (list[str]): Groundtruth output sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        loss (float): Loss value for a single sample.
    '''
    first_term = compute_crf_score(x, y, feature_dict)
    _, forward_score = compute_forward_score(x, feature_dict, states)
    
    loss = -(first_term - forward_score)
    return loss

# -45.37727
_, a = compute_forward_score("Great food with an awesome atmosphere !".split(), f, states)
a
# compute_forward_score(['hola', 'buenas', 'agradezco', 'a', 'todas', 'las', 'personas', 'que', 'hayan', 'puesto', 'estos', 'comentarios', 'porque', 'esto', 'la', 'verdad', 'te', 'deja', 'saber', 'lo', 'bueno', 'y', 'lo', 'malo', 'que', 'tiene', 'el', 'restaurante', 'pero', 'yo', 'recomiendo', 'a', 'todas', 'las', 'personas', 'este', 'restaurante', 'porque', 'muy', 'pocos', 'restaurantes', 'en', 'zaragoza', 'tienen', 'pruductos', 'que', 'tiene', 'el', 'puerto', 'como', 'por', 'ejemplo', 'el', 'pescado', 'es', 'de', 'muy', 'alta', 'calidad', 'y', 'la', 'carne', 'es', 'muy', 'buena', 'que', 'es', 'de', 'denominacion', 'de', 'origen', 'y', 'hay', 'platos', 'muy', 'sabrosos', 'y', 'lo', 'bueno', 'que', 'tiene', 'el', 'restaurante', 'es', 'que', 'prepara', 'sus', 'deleciosos', 'platos', 'al', 'momento', 'nunca', 'se', 'prepara', 'la', 'comida', 'y', 'luego', 'se', 'calienta', 'y', 'con', 'un', 'tiempo', 'justo', 'para', 'el', 'cliente', 'y', 'con', 'mucha', 'limpieza', 'en', 'cocina', 'y', 'con', 'mucho', 'cuidado', 'con', 'el', 'pescado', 'y', 'demas', 'pruductos', 'y', 'con', 'excelentes', 'postres', 'caseros', 'como', 'la', 'quesa', 'pasiega', ',', 'pudin', 'de', 'frutas', ',', 'bolas', 'de', 'melon', 'maceradas', 'en', 'vino', 'de', 'pedro', 'xmenez', ',', 'tarta', 'imperial', 'de', 'almendras', 'y', 'chocolate', ',', 'etc', '.'], f, states)

-45.37727632870497

We use the methods defined early to compute the loss across the entire dataset.

In [9]:
def get_dataset(path):
    '''
    Given a path, load the data from file.
    
    Inputs:
        path (Path object or str): path on the local directory to the dataset to perform inference on.
    Outputs:
        input_sequences (list[list[str]]): List of input sequences
        input_labels (list[list[str]]): List of input labels
    '''
    input_sequences = []
    input_labels = []
    
    # Read from dataset path
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        labels = []
        
        for line in lines:
            formatted_line = line.strip()
            
            # Not the end of sentence, add it to the list
            if len(formatted_line) > 0:
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]

                sentence.append(x)
                labels.append(y)
            else:
                # End of sentence
                input_sequences.append(sentence)
                input_labels.append(labels)
                sentence = []
                labels = []
    
    return input_sequences, input_labels

def compute_crf_loss(input_sequences, input_labels, feature_dict, states, nabla=0, regularization= False):
    '''
    Given a path, perform inference on sentences in the dataset and writes it to disk.
    
    Inputs:
        input_sequences (list[list[str]]): List of input sequences
        input_labels (list[list[str]]): List of input labels
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        loss (float): Loss value for the dataset.
    '''
    loss = 0
    for i in range(len(input_sequences)):
        sample_loss = compute_crf_loss_sample(input_sequences[i], input_labels[i], feature_dict, states)
        loss += sample_loss
    if regularization:
        reg_loss = 0
        for feature_key in feature_dict:
            reg_loss += feature_dict[feature_key]**2
        reg_loss = nabla*reg_loss
        loss += reg_loss
    return loss

# Part 3(ii)
Forward backward algorithm for computing gradients.

In [10]:
def compute_backward_score(x, feature_dict, states):
    '''
    Uses the backward algorithm to compute the score for a given sequence.
    
    Inputs:
        x (list[str]): Input sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        backward_score (float): Forward score for this sequence.
    '''
    n = len(x) # Number of words
    d = len(states) # Number of states
    backward_scores = np.zeros((n, d))
    
    # Start backward pass
    # Compute STOP transition scores
    for j, current_y in enumerate(states):
        transition_key = f"transition:{current_y}+STOP"
        transition_score = feature_dict.get(transition_key, -10**8)
        # Sum exponentials
        backward_scores[n-1, j] = transition_score

    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(n-1, 0, -1):
        for k, current_y in enumerate(states):
            temp_score = 0
            for j, next_y in enumerate(states):
                transition_key = f"transition:{current_y}+{next_y}"
                emission_key = f"emission:{next_y}+{x[i]}"
                
                transition_score = feature_dict.get(transition_key, -10**8)
                emission_score = feature_dict.get(emission_key, -10**8)

                # Sum exponentials
                temp_score += np.exp(min(emission_score + transition_score + backward_scores[i, j], 700))

            # Add to backward scores array
            backward_scores[i-1, k] = np.log(temp_score) if temp_score else -10**8
    
    # Compute for START
    backward_prob = 0
    for j, next_y in enumerate(states):
        transition_key = f"transition:START+{next_y}"
        emission_key = f"emission:{next_y}+{x[0]}" # Emission of last word

        transition_score = feature_dict.get(transition_key, -10**8)
        emission_score = feature_dict.get(emission_key, -10**8)
        # Sum exponentials
        overall_score = np.exp(min(emission_score + transition_score + backward_scores[0, j], 700))
        backward_prob += overall_score
    # End backward pass
    
    beta = np.log(backward_prob) if backward_prob else -700
    return backward_scores, beta

def forward_backward(x, feature_dict, states):
    '''
    Uses the forward-backward algorithm to compute the expected counts for a given sequence.
    
    Inputs:
        x (list[str]): Input sequence.
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        expected_counts (dict): Expected counts for each feature.
    '''
    
    n = len(x) # Number of words
    d = len(states) # Number of states
    
    forward_scores, alpha = compute_forward_score(x, feature_dict, states)
    forward_prob = np.exp(min(alpha, 700))
    backward_scores, beta = compute_backward_score(x, feature_dict, states)
    backward_prob = np.exp(min(beta, 700))
    
    # Computed expected counts for all features
    feature_expected_counts = defaultdict(float)
    
    # Compute expected emission counts
    for i in range(n):
        for j, current_y in enumerate(states):
            emission_key = f"emission:{current_y}+{x[i]}"
            feature_expected_counts[emission_key] += np.exp(min(forward_scores[i, j] + backward_scores[i, j] - alpha, 700))
    
    # Compute expected START / STOP transition counts
    for j, next_y in enumerate(states):
        start_transition_key = f"transition:START+{next_y}"
        feature_expected_counts[start_transition_key] += np.exp(min(forward_scores[0, j] + backward_scores[0, j] - alpha, 700))
        
        stop_transition_key = f"transition:{next_y}+STOP"
        feature_expected_counts[stop_transition_key] += np.exp(min(forward_scores[n-1, j] + backward_scores[n-1, j] - alpha, 700))
    
    # Compute expected transition probabilities for everything else
    for j, current_y in enumerate(states):
        for k, next_y in enumerate(states):
            transition_key = f"transition:{current_y}+{next_y}"
            transition_score = feature_dict.get(transition_key, -10**8)
            
            total = 0
            for i in range(n-1):
                emission_key = f"emission:{next_y}+{x[i+1]}"
                emission_score = feature_dict.get(emission_key, -10**8)

                total += np.exp(min(forward_scores[i, j] + backward_scores[i+1, k] + transition_score + emission_score - alpha, 700))

            feature_expected_counts[transition_key] = total
    
    return feature_expected_counts


0.7210505872358333

In [11]:
def get_feature_count(x, y, feature_dict):
    ''' 
    Inputs:
        x (list[str]): Complete input word sentence (without START or STOP tags)
        y (list[str]): Complete output label sequence
        feature_dict (dict[str] -> float): Dictionary that maps a given feature to its score.
    Outputs:
        p (float): Score given by p(y | x)
    '''
    
    # Input and output sequences must be of the same length
    assert(len(x) == len(y))
    n = len(x) # Sequence length
    
    # Stores the number of times each feature appears in (x, y)
    feature_count = defaultdict(int)
    
    # Compute emission features
    for i in range(n):
        formatted_word = x[i]
        emission_key = f"emission:{y[i]}+{formatted_word}"
        feature_count[emission_key] += 1
    
    # Compute transition features
    # Add START and STOP tags to y
    updated_y = ["START"] + y + ["STOP"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = f"transition:{prev_y}+{y_i}"
        feature_count[transition_key] += 1
    
    return feature_count

We wish to compute the gradient:
$\frac{\delta L(w)}{\delta \lambda_k} = \sum_i E_{p(y|x_i)} [ f_k (x_i, y) ] - \sum_i f_k (x_i, y_i) $

which can be computed by subtracting the actual counts of the features from the expected counts of each feature.

In [12]:
def compute_gradients(train_inputs, train_labels, f, states, nabla = 0, regularization = False):
    '''
    Uses the forward-backward algorithm to compute gradients analytically.
    
    Inputs:
        train_inputs (list[str]): Input sequence.
        train_labels (list[str]): Input labels.
        f (dict[str] -> float): Dictionary that maps a given feature to its score.
        states (list[str]): List of unique states.
    Outputs:
        backward_score (float): Backward score for this sequence.
        feature_gradients (dict[str] -> float): Dictionary that maps a given feature to its analytical gradient.
    '''
    
    feature_gradients = defaultdict(float)

    for i in range(len(train_labels)):
        x = train_inputs[i]
        y = train_labels[i]

        feature_expected_counts = forward_backward(x, f, states)
        actual_counts = get_feature_count(x, y, f)

        for k, v in feature_expected_counts.items():
            feature_gradients[k] += v

        for k, v in actual_counts.items():
            feature_gradients[k] -= v
        
    if regularization:
        for k, v in f.items():
            feature_gradients[k] += 2*nabla*f[k]

    return feature_gradients

We can check the results numerically:

In [13]:
states = get_states(data_dir / dataset / "train")
train_inputs, train_labels = get_dataset(data_dir / dataset / "train")

# Comment the below section out for Spanish, as O+the doesn't exist!
# Check against numerical gradient for several values
feature_key_checks = ['emission:O+the', 'transition:START+O', 'transition:O+O', 'transition:I-negative+I-positive']
feature_gradients = compute_gradients(train_inputs, train_labels, f, states)
loss1 = compute_crf_loss(train_inputs, train_labels, f, states)
delta = 1e-6

for feature_key in feature_key_checks:
    print("Checking", feature_key)
    new_f = f.copy()
    new_f[feature_key] += delta

    loss2 = compute_crf_loss(train_inputs, train_labels, new_f, states)

    numerical_gradient = (loss2 - loss1) / delta
    analytical_gradient = feature_gradients[feature_key]
    
    # Ensure numerical and analytical gradient are close
    assert(abs(numerical_gradient - analytical_gradient) / max(abs(numerical_gradient), 1e-8) <= 1e-3)

Checking emission:O+the
Checking transition:START+O
Checking transition:O+O
Checking transition:I-negative+I-positive


# Part 4(i)

These are helper functions for dictionary to array conversion

In [14]:
def numpy_to_dict(w, f, reverse = False):
    '''
    Converts a numpy array w to a dictionary with keys from f.
    '''
    for i,k in enumerate(f.keys()):
        f[k] = w[i]
    return f
def prepare_grad_for_bfgs(grads,f):
    '''
    Converts a dictionary to a numpy array.
    '''
    np_grads = np.zeros(len(f))
    for i,k in enumerate(f.keys()):
        np_grads[i] = grads[k]
    return np_grads

Running L-BFGS on the dataset

In [15]:
from scipy.optimize import fmin_l_bfgs_b 

states = get_states(data_dir / dataset / "train")
train_inputs, train_labels = get_dataset(data_dir / dataset / "train")

def callbackF(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    '''
    loss = compute_crf_loss(train_inputs,train_labels,f,states,0.1,regularization=True) 
    print('Loss:{0:.4f}'.format(loss))

def get_loss_grad(w,*args): 
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    Returns:
        loss: loss, float
        grads: gradients, numpy array
    '''

    train_inputs,train_labels,f,states = args
    f = numpy_to_dict(w,f)
    # compute loss and grad
    loss = compute_crf_loss(train_inputs, train_labels, f, states, 0.1, regularization=True)
    grads = compute_gradients(train_inputs, train_labels, f, states, 0.1, regularization=True)
    grads = prepare_grad_for_bfgs(grads, f) 
    # return loss and grad
    return loss, grads

init_w = np.zeros(len(f))
result = fmin_l_bfgs_b(get_loss_grad, init_w, args=(train_inputs, train_labels, f, states), pgtol=0.01, callback=callbackF)

Loss:19007.6578
Loss:12068.6971
Loss:11393.8658
Loss:10788.1557
Loss:10197.2511
Loss:9635.8611
Loss:8861.8972
Loss:8284.0767
Loss:8078.5501
Loss:7887.9522
Loss:7669.0499
Loss:7304.0618
Loss:6878.9758
Loss:6513.3204
Loss:6348.9892
Loss:6077.3882
Loss:5959.0012
Loss:5657.8769
Loss:5434.7521
Loss:5381.2663
Loss:5333.5624
Loss:5302.8242
Loss:5214.8213
Loss:5109.0913
Loss:4954.7691
Loss:4904.4185
Loss:4758.8482
Loss:4651.4197
Loss:4557.8960
Loss:4489.4024
Loss:4391.6532
Loss:4344.1131
Loss:4210.4801
Loss:4105.6601
Loss:4075.4750
Loss:4021.3417
Loss:3972.4631
Loss:3928.7447
Loss:3883.9005
Loss:3868.3786
Loss:3817.6166
Loss:3794.7724
Loss:3737.7940
Loss:3677.2720
Loss:3645.2902
Loss:3629.3420
Loss:3590.4545
Loss:3524.2284
Loss:3504.9559
Loss:3481.1172
Loss:3456.0795
Loss:3430.1995
Loss:3392.3634
Loss:3380.2429
Loss:3359.1886
Loss:3351.9971
Loss:3335.6018
Loss:3315.2665
Loss:3291.1134
Loss:3272.1581
Loss:3257.8409
Loss:3233.6885
Loss:3221.0961
Loss:3208.6069
Loss:3191.0252
Loss:3181.0028
Loss:

In [16]:
f = numpy_to_dict(result[0], f)

In [17]:
inference(data_dir / dataset / "dev.in", states, f, "p4")